# Pad Cropper
- Date modified: 20200728_1210
    


In [1]:
### Library 
import pandas as pd
from glob import glob
import os
from tqdm import tqdm
import random 

import numpy as np
import cv2

In [27]:
class Rect():
    
    def __init__(self, x, y, w, h, label):
        self.x=x;
        self.y=y
        self.w=w
        self.h=h
        self.label=label
        
        self.cx = self.x + self.w /2
        self.cy = self.y + self.h /2
        
    def set_x(self, x):
        self.w = self.w - (x - self.x)
        self.x=x
        self.cx = self.x + self.w /2
        
    def set_y(self, y):
        self.h = self.h - (y - self.x)
        self.y=y
        self.cy = self.y + self.h /2
        
    def set_br_x(self, x):
        self.w = self.w + x - self.x
        self.cx = self.x + self.w /2
        
    def set_br_y(self, y):
        self.h = self.h + y - self.h
        self.cy = self.y + self.h /2
        

    def br(self):
        return {"x": self.x+self.w, "y": self.y+self.h}
    
    def __str__(self):
        return "{},{},{},{}".format(self.x, self.y, self.w, self.h)
    

    def floor(self):
        self.x = int(self.x)
        self.y = int(self.y)
        self.w = int(self.w)
        self.h = int(self.h)

### Search areas

In [29]:
def roi_generator(refdes_list,  k_nearest = 5):

    def get_DistanceMatrix(refdes_list):
        x1 = np.zeros([len(refdes_list),len(refdes_list)])
        for i, r in enumerate(refdes_list):
            x1[i] = r.cx
        x2 = np.transpose(x1)

        y1 = np.zeros([len(refdes_list),len(refdes_list)])
        for i, r in enumerate(refdes_list):
            y1[i] = r.cy
        y2 = np.transpose(y1)
        dist = np.sqrt( np.square(x2-x1) + np.square(y2-y1))
        return dist
    
    
    def get_groups(refdes_list, k_nearest = 5):
        groups=[]
        names=[]
        dist=get_DistanceMatrix(refdes_list)
        skip=0
        
        for i, r in enumerate(refdes_list):
            sorted_idxs = np.sort(np.argsort(dist [i])[:k_nearest])
            name = ""
            for e in sorted_idxs: name+=str(e)
            if name not in names:
                group=[]
                for sorted_idxs in sorted_idxs:
                    group.append(refdes_list[sorted_idxs])
                groups.append(group)    
                names.append(name)
            else:
                skip = skip + 1
           
        #print("len(groups):\t",len(groups))
        #print("skipped ", skip)
        return names, groups
    
    def get_searchareas(refdes_list, k_nearest):
        search_areas=[]
        names, groups=get_groups(refdes_list, k_nearest)
        
        for group in groups:
            xmin = group[0].x
            ymin = group[0].y

            xmax = group[0].x
            ymax = group[0].y
            for e in group:
                xmin = xmin if xmin<e.x else e.x
                ymin = ymin if ymin<e.y else e.y
                xmax = xmax if xmax>e.br()['x'] else e.br()['x']
                ymax = ymax if ymax>e.br()['y'] else e.br()['y']
                
                #Random padding
                xmin = xmin - random.uniform(5, 30)
                ymin = ymin - random.uniform(5, 30)
                xmax = xmax + random.uniform(5, 30)
                ymax = ymax + random.uniform(5, 30)

            search_areas.append(Rect (xmin, ymin, xmax-xmin, ymax-ymin, 'search_area'))
        return names, search_areas
    
        
    
    ########################
    # MAIN
    ########################
    def main():
        
        # Generated search areas
        names, search_areas = get_searchareas(refdes_list, k_nearest)
        #print("Generated search areas:\t{}".format(len(search_areas)))
        
        # Return
        return names, search_areas

    
    return main()

### Read files

In [30]:
board_dpaths = glob("D:/FZ_WS/JyNB/Yolo_Pad_WS/Yolo_Pad_00/data/dataset/A-Dataset-01/Pad Detection/Dataset/Good Dataset/*")
for board_fpath in board_dpaths:
    board_fpath=os.path.normpath(board_fpath)
    fpaths = glob(board_fpath+"/*.txt")
    board_name = os.path.basename(board_fpath)
    sa_per_board=0
    print("Board name:\t", board_name)
    
    limit = 0
    for fpath in tqdm(fpaths):
        
        #Read Label file
        refdes=[]    
        with open(fpath, 'r') as f:
            lines = f.readlines()
            for line in lines:
                if (line.startswith('r')):
                    label = line.strip().upper().split(',')
                    #['R', '0', 'PAD', '923', '861', '35', '35']
                    refdes.append(Rect( int(label[3]),  int(label[4]), int(label[5]),  int(label[6]),  label[2]))
        #print("Found {} refdes".format(len(refdes)))

        
        #Search areas
        names, search_areas = [], []
        for k in range(2, 10):
            cnames, csearch_areas = roi_generator(refdes,  k_nearest = k)
            names.extend(cnames)
            search_areas.extend(csearch_areas)
            #print("{} R:\tSA:\t{}".format(k, len(csearch_areas)))
        
        #Unique Combination
        unames=set(names)
        sa_per_board=sa_per_board+len(unames)
        #if (len(unames)!=len(names)): print("ALERT!! len(unames)!=len(names): {}!={}".format(len(unames),len(names)) )

        
        # Save 
        img_fpath = fpath.replace("txt", "jpg").replace("_Label","")
        img_name = os.path.basename(img_fpath)
        full_image = cv2.imread(img_fpath)
        h, w, _ = full_image.shape
        for uname in unames:
            i = names.index(uname)
            search_area=search_areas[i]
            if (os.path.isfile(img_fpath)):
                # Search area Images
                out_fpath = "D:/FZ_WS/JyNB/Yolo_Pad_WS/Yolo_Pad_00/scripts/label/Out/"+board_name+"-"+img_name[:-4]+"-{}.jpg".format(i)
                
                #Realign
                if (search_area.x<0): search_area.set_x(0)
                if (w < search_area.br()['x'] ): search_area.set_br_x(w)
                if (search_area.y<0): search_area.set_y(0)
                if (h < search_area.br()['y'] ): search_area.set_br_y(h)
                    
                #Image
                search_area.floor()
                img = full_image[search_area.y:search_area.br()['y'], search_area.x:search_area.br()['x']]
                cv2.imwrite(out_fpath, img)

                # Search area label
                out_fpath=out_fpath.replace(".jpg", ".txt")
                with open(out_fpath, 'w') as f:
                    for r in refdes:
                        isOverlapping1D = lambda xmin1, xmax1, xmin2, xmax2 : xmax1 >= xmin2 and xmax2 >= xmin1
                        isOverlapping   = isOverlapping1D(r.x, r.br()['x'], search_area.x, search_area.br()['x']) and  isOverlapping1D(r.y, r.br()['y'], search_area.y, search_area.br()['y'])
                        
                        if(isOverlapping):
                            if (r.x < search_area.x): continue
                            if (r.y < search_area.y): continue
                            if (r.br()['x']>search_area.br()['x']): continue
                            if (r.br()['y']>search_area.br()['y']): continue
                            f.write("{},{},{},{},0\n".format(r.x-search_area.x, r.y-search_area.y, r.br()['x']-search_area.x,r.br()['y']-search_area.y))
            else:
                    print("Error: File not found", os.path.isfile(img_fpath))
        break
        if (limit > 50 ): break
        limit = limit +1
    print("Search areas total:\t", sa_per_board)
    break
print("Completed!")

  0%|                                                                                                                                                                               | 0/199 [00:00<?, ?it/s]

Board name:	 49043BottomPackage
618,529,240,393
567,743,309,252
663,372,798,265
591,429,276,330
686,864,149,1024
716,789,640,1024
650,653,758,1024
584,131,922,587
574,312,944,505
631,637,213,242
650,437,185,229
634,619,255,325
627,690,205,264
610,667,859,1024
942,663,209,1024
605,333,399,343
598,71,924,549
567,67,1012,641
625,633,813,1024
685,924,113,95
638,675,185,170
695,377,702,200
654,775,761,1024
570,533,309,470
637,712,206,212
656,665,154,102
611,856,141,1024
637,338,372,269
589,248,328,607
641,480,171,170
621,780,192,192
632,620,185,325
611,359,251,320
638,355,170,247
563,313,341,532
641,113,887,683
604,835,251,1024
658,123,783,461
622,528,216,376
519,610,437,1024
529,86,1045,582
627,680,223,290
574,247,1024,1024
589,771,195,1024
617,646,877,1024
625,53,865,528
600,664,272,331
603,154,975,543
628,677,201,253
856,129,363,220
607,387,241,285
637,455,219,243
670,883,126,107
553,124,445,457
797,147,172,231
662,141,756,446
600,95,404,290
644,131,842,675
551,471,379,528
594,389,263,27

  0%|                                                                                                                                                                               | 0/199 [00:00<?, ?it/s]

Search areas total:	 160
Completed!
